In [ ]:
%set_env LD_LIBRARY_PATH=llama.cpp/ggml/src:llama.cpp/src
import threading
import subprocess

In [ ]:
# pkill -9 llama-server
subprocess.run(['pkill', '-9', 'llama-server'])

In [ ]:
model = "Llama-3.2-1B-Instruct"
# model = "Meta-Llama-3.1-8B-Instruct"
# model = "Qwen2.5-1.5B-Instruct"
# model = "gemma-2-2b-it"
# model = "Phi-3.5-mini-instruct"
# model = "Phi-3-medium-128k-instruct"
# model = "Mistral-Nemo-Instruct-2407"
# model = "Mistral-Small-Instruct-2409"
# size = "IQ4_XS"
# size = "Q5_K_M"
size = "Q8_0"

## MMLU-Pro

In [ ]:
# llama-server -m models/gemma2/gemma-2-2b-it-Q4_K_S.gguf --ctx-size 6144 --n-gpu-layers 99 --flash-attn --parallel 2
ctx_size = '6144' # 2*3072
parallel = '2'
ngl = '99'

server = threading.Thread(target=lambda: subprocess.run([
    'llama.cpp/bin/llama-server',
    '-m', "models/" + model + "-" + size + ".gguf",
    '--ctx-size', ctx_size,
    '--n-gpu-layers', ngl,
    '--parallel', parallel,
    '--flash-attn'], check=True), daemon=True)

server.start()

In [ ]:
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset sam-paech/mmlu-pro-nomath-sml --parallel 2 --style multi_chat_zeroshot
style = 'multi_chat_zeroshot'
mmlu_pro_nomath = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'sam-paech/mmlu-pro-nomath-sml',
    '--parallel', parallel,
    '--style', style], check=True), daemon=True)

mmlu_pro_nomath.start()

In [ ]:
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset TIGER-Lab/MMLU-Pro --category 'computer science' --parallel 2 --style multi_chat_zeroshot
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset TIGER-Lab/MMLU-Pro --category 'engineering' --parallel 2 --style multi_chat_zeroshot
style = 'multi_chat_zeroshot'
category = "computer science"
# category = "engineering"
mmlu_pro_category = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'TIGER-Lab/MMLU-Pro',
    '--parallel', parallel,
    '--category', category,
    '--style', style], check=True), daemon=True)

mmlu_pro_category.start()

## openai/MMMLU

In [ ]:
ctx_size = '8700' # 3*2900
parallel = '3'
ngl = '99'

server = threading.Thread(target=lambda: subprocess.run([
    'llama.cpp/bin/llama-server',
    '-m', "models/" + model + "-" + size + ".gguf",
    '--ctx-size', ctx_size,
    '--n-gpu-layers', ngl,
    '--parallel', parallel,
    '--flash-attn'], check=True), daemon=True)

server.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category ES_LA
mmmlu_es = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'ES_LA'], check=True), daemon=True)

mmmlu_es.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category FR_FR
mmmlu_fr = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'FR_FR'], check=True), daemon=True)

mmmlu_fr.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category ZH_CN
mmmlu_zh = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'ZH_CN'], check=True), daemon=True)

mmmlu_zh.start()

In [ ]:
### ATTENTION: NOT include in our customized benchmark for saving time
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category DE_DE
mmmlu_de = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'DE_DE'], check=True), daemon=True)

mmmlu_de.start()

In [ ]:
### ATTENTION: NOT include in our customized benchmark for saving time
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category PT_BR
mmmlu_pt = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://localhost:8080/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'PT_BR'], check=True), daemon=True)

mmmlu_pt.start()